In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('dataset.csv')
print(df.head())  

                           id  year  month  day  hour  minute  press  humid  \
0         2021-09-16 18:00:00  2021      9   16    18       0   1012     78   
1  2021-09-16 18:10:00.000004  2021      9   16    18      10   1012     78   
2  2021-09-16 18:19:59.999999  2021      9   16    18      19   1012     78   
3  2021-09-16 18:30:00.000003  2021      9   16    18      30   1012     79   
4  2021-09-16 18:39:59.999998  2021      9   16    18      39   1012     79   

   temp  ws40  ...  vertdisp140  vertdisp150  vertdisp160  vertdisp170  \
0  27.6  7.44  ...         0.44         0.44         0.44         0.45   
1  27.5  7.45  ...         0.48         0.50         0.51         0.53   
2  27.5  7.59  ...         0.33         0.35         0.37         0.38   
3  27.5  7.64  ...         0.47         0.46         0.47         0.48   
4  27.4  9.06  ...         0.38         0.38         0.36         0.35   

   vertdisp180  vertdisp190  vertdisp200  vertdisp220  vertdisp240  \
0         

In [3]:
df.index = pd.to_datetime(df['id'], errors='ignore')
df.index = df['id'].apply(lambda x: pd.to_datetime(str(x).split('.')[0], errors='ignore'))

C:\Users\lucas\AppData\Local\Temp\ipykernel_1552\2288332652.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df.index = pd.to_datetime(df['id'], errors='ignore')
C:\Users\lucas\AppData\Local\Temp\ipykernel_1552\2288332652.py:2: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  df.index = df['id'].apply(lambda x: pd.to_datetime(str(x).split('.')[0], errors='ignore'))


In [4]:

cols = list(df)[6:9]
cols += ["ws100", "wdir100", "cis6", "cis7", "wdisp100"]


cols

['press', 'humid', 'temp', 'ws100', 'wdir100', 'cis6', 'cis7', 'wdisp100']

In [5]:
df_for_training = df[cols].astype(float)

df_for_training

,press,humid,temp,ws100,wdir100,cis6,cis7,wdisp100
id,,,,,,,,
2021-09-16 18:00:00,1012.0,78.0,27.6,8.51,53.95,0.015,0.014,1.16
2021-09-16 18:10:00,1012.0,78.0,27.5,8.64,52.19,0.008,0.008,1.04
2021-09-16 18:19:59,1012.0,78.0,27.5,8.65,52.02,0.010,0.010,1.04
2021-09-16 18:30:00,1012.0,79.0,27.5,8.86,53.65,0.017,0.016,0.96
2021-09-16 18:39:59,1012.0,79.0,27.4,10.00,53.49,0.007,0.007,0.72
...,...,...,...,...,...,...,...,...
2021-11-08 13:49:59,1008.0,66.0,30.6,8.36,41.55,0.001,0.002,1.35
2021-11-08 14:00:00,1007.0,66.0,30.8,8.62,37.31,0.005,0.004,1.23
2021-11-08 14:09:59,1007.0,67.0,30.7,8.96,37.27,0.001,0.003,1.24


In [6]:
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)


df_for_training_scaled

array([[ 1.24663888,  0.63089718, -0.47170478, ...,  0.59346353,
         0.63140266,  0.29884617],
       [ 1.24663888,  0.63089718, -0.53615641, ..., -0.20223613,
        -0.09126084, -0.05423555],
       [ 1.24663888,  0.63089718, -0.53615641, ...,  0.02510663,
         0.14962699, -0.05423555],
       ...,
       [-1.83954951, -0.91490778,  1.52629574, ..., -0.9979358 ,
        -0.69348042,  0.53423398],
       [-1.83954951, -0.91490778,  1.33294085, ..., -0.65692166,
        -1.2957    ,  0.41654007],
       [-1.83954951, -0.77438006,  1.13958596, ..., -1.11160718,
        -0.5730365 , -0.05423555]])

In [7]:
sequence_length = 48  
n_features = df_for_training_scaled.shape[1] 

In [8]:
X, y = [], []

for i in range(len(df_for_training_scaled) - sequence_length):
    X.append(df_for_training_scaled[i:i + sequence_length])
    y.append(df_for_training_scaled[i + sequence_length, cols.index("ws100")])

X = np.array(X)
y = np.array(y)

print(f"Formato de X: {X.shape}")  
print(f"Formato de y: {y.shape}")  

Formato de X: (7513, 48, 8)
Formato de y: (7513,)


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

X_train: (6010, 48, 8), X_test: (1503, 48, 8)
y_train: (6010,), y_test: (1503,)


In [10]:
model = Sequential()

model.add(LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20


c:\Users\lucas\Projetos\TFG_VENTO_SUL\TFG-Vento-Sul\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


188/188 ━━━━━━━━━━━━━━━━━━━━ 12s 55ms/step - loss: 0.5709 - val_loss: 0.1979
Epoch 2/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 53ms/step - loss: 0.1695 - val_loss: 0.1331
Epoch 3/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 10s 55ms/step - loss: 0.1596 - val_loss: 0.1219
Epoch 4/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 9s 47ms/step - loss: 0.1592 - val_loss: 0.1594
Epoch 5/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - loss: 0.1444 - val_loss: 0.1375
Epoch 6/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 22s 115ms/step - loss: 0.1507 - val_loss: 0.1413
Epoch 7/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 22s 116ms/step - loss: 0.1570 - val_loss: 0.1207
Epoch 8/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 21s 113ms/step - loss: 0.1467 - val_loss: 0.1188
Epoch 9/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 21s 113ms/step - loss: 0.1452 - val_loss: 0.1196
Epoch 10/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 21s 113ms/step - loss: 0.1504 - val_loss: 0.1144
Epoch 11/20
188/188 ━━━━━━━━━━━━━━━━━━━━ 21s 114ms/step - loss: 0.1428 - val_loss: 0.1238
Epoch 12/20
188/188 ━━━━━━━━━━━━━━━━

In [11]:
y_pred = model.predict(X_test)

n_amostras = len(y_pred)
n_features = df_for_training.shape[1]  

y_pred_full = np.zeros((n_amostras, n_features))
y_pred_full[:, 3] = y_pred.flatten() 

y_pred_orig = scaler.inverse_transform(y_pred_full)[:, 3]  

47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step


In [12]:
y_test_full = np.zeros((len(y_test), n_features))
y_test_full[:, 3] = y_test.flatten() 

y_test_orig = scaler.inverse_transform(y_test_full)[:, 3]

In [3]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import numpy as np


rmse_manual = np.sqrt(np.mean((y_train - y_pred) ** 2))
print(f"RMSE (manual): {rmse_manual}")

rmse_sklearn = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"RMSE (sklearn): {rmse_sklearn}")

plt.figure(figsize=(10, 6))
plt.plot(y_test_orig, label='Valor Real (ws100)')
plt.plot(y_pred_orig, label='Previsão (ws100)')
plt.legend()
plt.show()

NameError: name 'y_train' is not defined